# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


Previously, we did some exploratory data analysis by visualizing the SpaceX launch dataset using `matplotlib` and `seaborn` and we discovered some preliminary correlations between the launch site and success rates. Here, we will be displaying interactive geospacial visual analytics using `Folium`.


## Objectives


*   Mark all launch sites on a map
*   Mark the success/failed launches for each site on the map
*   Calculate the distances between a launch site to its proximities

Through these visualizations, we should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip3 install folium
!pip3 install wget

In [14]:
# importing all dependencies
import folium
import wget
import pandas as pd

from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

## Marking all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [13]:
# Download and read the `spacex_launch_geo.csv` (if needed: csv file is included in the repository folders)
# spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')

# Import the file from folder
spacex_csv_file = 'Data/spacex_launch_geo.csv'

spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [3]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610746


The above coordinates are just plain numbers that can not give us any intuitive insights about where those launch sites are. If one is used to dealing with geographic coordinates, one could interpret those numbers directly in their mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first start by creating a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas. This first close up is for demonstrative purposes only.


In [4]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We can then use a `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate, as follows:


In [5]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#008eed', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#008eed;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


In [6]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)

def add_launch_sites(row):
    # For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
    circle = folium.Circle([launch_sites_df['Lat'][row],launch_sites_df['Long'][row]], radius=50, color='#d35400', fill=True).add_child(folium.Popup(launch_sites_df['Launch Site'][row]))
    marker = folium.map.Marker(
        [launch_sites_df['Lat'][row],launch_sites_df['Long'][row]],
        # Create an icon as a text label
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % launch_sites_df['Launch Site'][row],
            )
        )
    site_map.add_child(circle)
    site_map.add_child(marker)

for row in range(len(launch_sites_df)):
    add_launch_sites(row)
    
site_map

Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Marking the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame `spacex_df` has detailed launch records, and the `class` column indicates if this launch was successful or not


In [8]:
# for each row in spacex_df data frame we create a Marker object with its coordinate
for index, record in spacex_df.iterrows():
    marker = folium.map.Marker([record[1],record[2]],icon=folium.Icon(color='white',icon_color=record[4],icon='rocket',prefix='fa'))
    marker_cluster.add_child(marker)

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


In order to do this we first create a `MarkerCluster` object, then create a new column in the `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value for each launch.


In [25]:
marker_cluster = MarkerCluster()


In [7]:
# create marker cluster object
marker_cluster = MarkerCluster()

# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)


,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [12]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


Finally, for each launch result in the `spacex_df` data frame, we add a `folium.Marker` to `marker_cluster`. This will display all launches at their respective site and color coded according to their final outcome. It should be relatively simple to have an idea of the launch site's average success rate based on these markers.

In [13]:
# Add marker_cluster to current site_map
#site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    marker = folium.map.Marker([record[1],record[2]],icon=folium.Icon(color='white',icon_color=record[4],prefix='fa',icon='rocket'))
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)


# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

site_map

# Calculate the distances between a launch site to its proximities


Next, we want to explore and analyze the proximities of the launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as a railway or city).


In [9]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position = 'topright',
    separator = ' Long: ',
    empty_string ='NaN',
    lng_first = False,
    num_digits = 20,
    prefix = 'Lat:',
    lat_formatter = formatter,
    lng_formatter = formatter,
)

site_map.add_child(mouse_position)
site_map

Your updated map with distance line should look like the following screenshot:


One can now zoom in to a launch site and explore its proximity to see if we can easily find any railways, highways, coastline, etc. Feel free to move your mouse to these points and mark down their coordinates (shown on the top-left) in order to calculate these features' distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [10]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [ ]:
# Create a marker with distance to a closest city, coastline, highway, etc.
# Draw a line between the marker to the launch site


*TODO:* Mark down a point on the closest railway using MousePosition and calculate the distance between the railway point to the launch site.


In [16]:
# find coordinate of railway point
railway_lat, railway_lon = 28.57295, -80.65392
launch_site_lat, launch_site_lon = launch_sites_df['Lat'][2], launch_sites_df['Long'][2]
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [17]:
# Create and add a folium.Marker on your selected closest railway point on the map
# Display the distance between railway point and launch site using the icon property 
# for example
distance_marker = folium.Marker(
    [railway_lat,railway_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway),
        )
    )

*TODO:* Draw a `PolyLine` between a launch site to the selected


In [18]:
# Create a `folium.PolyLine` object using the railway point coordinate and launch site coordinate
lines=folium.PolyLine([(railway_lat,railway_lon),(launch_site_lat,launch_site_lon)], weight=1)
site_map.add_child(lines)

*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, coastline, highway, etc.


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.
